In [79]:
import torch
from torch import nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']
plt.style.use('fivethirtyeight')

In [4]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return (Y)

x = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
k = torch.tensor([[0, 1], [2, 3]])

print (x, x.shape)
print (k, k.shape)

print (corr2d(x, k))

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]]) torch.Size([3, 3])
tensor([[0, 1],
        [2, 3]]) torch.Size([2, 2])
tensor([[19., 25.],
        [37., 43.]])


In [5]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return (Y)

class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return (corr2d(x, self.weight) + self.bias)


In [6]:
X = torch.ones((6, 8))
X[:,  2:6] = 0
print (X, X.shape)

# K representing finite difference operator: edge detection
K = torch.tensor([[1.0, -1.0]])
print (K, K.shape)

Y = corr2d(X, K)

print (Y, Y.shape)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]]) torch.Size([6, 8])
tensor([[ 1., -1.]]) torch.Size([1, 2])
tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]) torch.Size([6, 7])


In [7]:
corr2d(X.T, K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [44]:
conv2D = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8))

print('Original X:')
print(X, X.shape)

print('Original Y to fit:')
Y = Y.reshape([1, 1, 6, 7])
print(Y, Y.shape)

print('Example of Conv2D:')
Z = conv2D(X)
print(Z, Z.shape)


Original X:
tensor([[[[1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.]]]]) torch.Size([1, 1, 6, 8])
Original Y to fit:
tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]]) torch.Size([1, 1, 6, 7])
Example of Conv2D:
tensor([[[[ 0.0917, -0.3280,  0.0000,  0.0000,  0.0000,  0.4197,  0.0917],
          [ 0.0917, -0.3280,  0.0000,  0.0000,  0.0000,  0.4197,  0.0917],
          [ 0.0917, -0.3280,  0.0000,  0.0000,  0.0000,  0.4197,  0.0917],
          [ 0.0917, -0.3280,  0.0000,  0.0000,  0.0000,  0.4197,  0.0917],
          [ 0.0917, -0.3280,  0.0000,  0.0000,  

/Users/jsmidt/Library/Python/3.8/lib/python/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [45]:
lr = 0.03
for i in range(12):
    Y_hat = conv2D(X)
    l = ((Y_hat - Y)**2).sum()
    conv2D.zero_grad()
    l.backward()
    #print(conv2D.weight.grad)
    conv2D.weight.data[:] -= lr * conv2D.weight.grad
    if (i+1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.item():.3f}')

epoch 2, loss 9.358
epoch 4, loss 1.589
epoch 6, loss 0.275
epoch 8, loss 0.049
epoch 10, loss 0.010
epoch 12, loss 0.002


In [48]:
print (conv2D.weight.data.view(2))
print (Y)
print (Y_hat)

tensor([ 0.9967, -0.9904])
tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]])
tensor([[[[-0.0079,  0.9859,  0.0000,  0.0000,  0.0000, -0.9938, -0.0079],
          [-0.0079,  0.9859,  0.0000,  0.0000,  0.0000, -0.9938, -0.0079],
          [-0.0079,  0.9859,  0.0000,  0.0000,  0.0000, -0.9938, -0.0079],
          [-0.0079,  0.9859,  0.0000,  0.0000,  0.0000, -0.9938, -0.0079],
          [-0.0079,  0.9859,  0.0000,  0.0000,  0.0000, -0.9938, -0.0079],
          [-0.0079,  0.9859,  0.0000,  0.0000,  0.0000, -0.9938, -0.0079]]]],
       grad_fn=<ConvolutionBackward0>)


In [75]:
X = torch.zeros((6,8))

X[1:6,0:8].fill_diagonal_(1.0)
X[0:6,0:8].fill_diagonal_(1.0)
X[2:6,0:8].fill_diagonal_(1.0)

X

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0.]])

In [78]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return (Y)


# K representing finite difference operator: edge detection
K = torch.tensor([[1.0, -1.0]])
print (K, K.shape)

print ("Normal X")
Y = corr2d(X, K)
print (Y)

print ("Transpose X")
Y = corr2d(X.T, K)
print (Y)

print ("Transpose K")
Y = corr2d(X, K.T)
print (Y)

tensor([[ 1., -1.]]) torch.Size([1, 2])
Normal X
tensor([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [-1.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0., -1.,  0.,  0.,  1.,  0.]])
Transpose X
tensor([[ 0.,  0.,  1.,  0.,  0.],
        [-1.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  1.],
        [ 0.,  0., -1.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  0.],
        [ 0.,  0.,  0.,  0., -1.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]])
Transpose K
tensor([[ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0., -1.,  0.,  0.]])


In [22]:
def comp_conv2d(conv2D, X):
    X = X.reshape((1,1) + X.shape)
    Y = conv2D(X)
    return Y.reshape(Y.shape[2:])

conv2D = nn.LazyConv2d(1, kernel_size=3, padding=1)
X = torch.rand((8,8))
print (X.shape)
Y = comp_conv2d(conv2D, X)

print (Y, Y.shape)


torch.Size([8, 8])
tensor([[ 0.2249,  0.0190, -0.0297,  0.3708,  0.1563,  0.1599, -0.1715, -0.0242],
        [ 0.1242, -0.1107, -0.0125, -0.2845, -0.1606, -0.2476, -0.3285, -0.2991],
        [ 0.2812, -0.2579, -0.2680, -0.3600, -0.0043, -0.0227,  0.0208, -0.3025],
        [-0.1561, -0.2783,  0.0342,  0.1681,  0.3364, -0.0629, -0.2515, -0.3790],
        [-0.0200,  0.3406,  0.0017, -0.1172, -0.3205, -0.2876, -0.5326, -0.3680],
        [ 0.1894, -0.3008, -0.3717, -0.4712, -0.3533, -0.2045, -0.1415, -0.2119],
        [-0.0662, -0.2230, -0.1037,  0.0741, -0.1633,  0.0549,  0.0603,  0.0845],
        [-0.3636, -0.2313, -0.0654, -0.3774, -0.1489, -0.2385, -0.0418, -0.4194]],
       grad_fn=<ReshapeAliasBackward0>) torch.Size([8, 8])


In [39]:
conv2D = nn.LazyConv2d(1, kernel_size=3, padding=1, stride=2)

Y = comp_conv2d(conv2D, X)

print (Y, Y.shape)

tensor([[0.4322, 0.5679, 0.2444, 0.4230],
        [0.5999, 0.8545, 0.8634, 0.9430],
        [0.6543, 0.3259, 0.5415, 0.6712],
        [0.5548, 0.7564, 0.3921, 0.4204]], grad_fn=<ReshapeAliasBackward0>) torch.Size([4, 4])


In [43]:
conv2D = nn.LazyConv2d(1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2D, X)


/Users/jsmidt/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Parameter containing:
tensor([[[[-0.1666,  0.0273, -0.1168,  0.1210,  0.0513],
          [-0.1438, -0.0706,  0.2099,  0.1201, -0.0334],
          [ 0.2075,  0.1467,  0.1505, -0.0046,  0.0119]]]], requires_grad=True)

In [80]:
def comp_conv2d(conv2D, X):
    X = X.reshape((1,) + X.shape)
    Y = conv2D(X)
    return Y.reshape(Y.shape[1:])


conv2D = nn.LazyConv2d(3, kernel_size=1)
X = torch.rand((3, 3, 3))
print(X,X.shape)
Y = comp_conv2d(conv2D, X)
print(Y, Y.shape)


tensor([[[0.3447, 0.1833, 0.0658],
         [0.8468, 0.2135, 0.4940],
         [0.7293, 0.5813, 0.0133]],

        [[0.6210, 0.2011, 0.8367],
         [0.0631, 0.6373, 0.0665],
         [0.2933, 0.5737, 0.5621]],

        [[0.8011, 0.2778, 0.6255],
         [0.4148, 0.5092, 0.2928],
         [0.5838, 0.8066, 0.2257]]]) torch.Size([3, 3, 3])
tensor([[[ 0.2678,  0.2477,  0.1635],
         [ 0.4945,  0.2316,  0.3682],
         [ 0.4340,  0.3577,  0.1728]],

        [[-0.4817, -0.3961, -0.5909],
         [-0.2504, -0.5127, -0.3087],
         [-0.3317, -0.4312, -0.5278]],

        [[ 0.3482,  0.3519,  0.4249],
         [ 0.2654,  0.3950,  0.3056],
         [ 0.2920,  0.3228,  0.4304]]], grad_fn=<ReshapeAliasBackward0>) torch.Size([3, 3, 3])


In [74]:
Y.shape[1:]

torch.Size([2, 3, 3])

In [55]:
(1,) + X.shape

(1, 2, 2, 2)

In [87]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i:i+p_h, j: j+p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i:i+p_h, j: j+p_w].mean()

    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])

X = torch.arange(9).view(3,3).float()

print (X)

print (pool2d(X, (2,2)))
print (pool2d(X, (2,2), mode='avg'))

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])
tensor([[4., 5.],
        [7., 8.]])
tensor([[2., 3.],
        [5., 6.]])


In [89]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))

pool2d = nn.MaxPool2d(3)

pool2d(X)

tensor([[[[10.]]]])

In [91]:
pool2d = nn.MaxPool2d((2,3), padding=(0,1), stride=(2,3))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [92]:
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [93]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

In [94]:
# Random pooling can improve accuracy in some cases. 